# What happens:
File is created and email is prepared

## Import 

In [ ]:
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from sqlalchemy import create_engine
import win32com.client as win32

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [ ]:
directory = os.path.abspath(os.getcwd()) + "/"
rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

### Make connection with Oracle DB

In [ ]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
Also see how much time it took

In [ ]:
%%time
d = pd.read_sql_query("select * from v_rzb_all_contracts where book_code like ('RBUK%')", connection_RAY)

### Save var in an excel file

In [ ]:
ex_folder = "Export/"

# determining the name of the file
file_name = 'RBUK_' + rd + '_all_contracts.xlsx'

# saving the excel
d.to_excel(ex_folder + file_name, index=False)
print('DataFrame is written to Excel File successfully.')

### Send the email 

In [ ]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'bernadette.dettmer@uk.rbinternational.com; devon.chaplin@uk.rbinternational.com'
mail.Cc = ''
mail.Subject = 'RBUK Ray Data ' + rd
mail.Body = '''Dear Bernadette,

Please find attached, the RBUK data for ''' + rd +'''

Kr
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

print(directory + ex_folder + file_name)
# To attach a file to the email (optional):
attachment  = directory + ex_folder + file_name
mail.Attachments.Add(attachment)

mail.Display()